# Topic Analysis
* Here, I have explored text data present in the dataset in form for email subject and content
* I have used preprocessed corpus/dataframe created from previous exercise (refer:enron_emails_analysis notebook)

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving message_df_all.pkl to message_df_all.pkl
User uploaded file "message_df_all.pkl" with length 172842594 bytes


In [0]:
# necesaary lib import
import pandas as pd
from gensim.corpora.dictionary import Dictionary
import pandas as pd 
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from gensim.models import LdaModel

In [0]:
emails_df = pd.read_pickle('message_df_all.pkl')

In [4]:
emails_df.head()

,Cc,X-cc,From,Subject,X-Folder,X-bcc,Bcc,content,To,X-Origin,X-FileName,X-From,Date,X-To,Message-ID,user
0,None,,phillip.allen@enron.com,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",,None,Here is our forecast\n\n,tim.belden@enron.com,Allen-P,pallen (Non-Privileged).pst,Phillip K Allen,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Tim Belden <Tim Belden/Enron@EnronXGate>,<18782981.1075855378110.JavaMail.evans@thyme>,allen-p
1,None,,phillip.allen@enron.com,Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",,None,Traveling to have a business meeting takes the...,john.lavorato@enron.com,Allen-P,pallen (Non-Privileged).pst,Phillip K Allen,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",John J Lavorato <John J Lavorato/ENRON@enronXg...,<15464986.1075855378456.JavaMail.evans@thyme>,allen-p
2,None,,phillip.allen@enron.com,Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,,None,test successful. way to go!!!,leah.arsdall@enron.com,Allen-P,pallen.nsf,Phillip K Allen,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Leah Van Arsdall,<24216240.1075855687451.JavaMail.evans@thyme>,allen-p
3,None,,phillip.allen@enron.com,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,,None,"Randy,\n\n Can you send me a schedule of the s...",randall.gay@enron.com,Allen-P,pallen.nsf,Phillip K Allen,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",Randall L Gay,<13505866.1075863688222.JavaMail.evans@thyme>,allen-p
4,None,,phillip.allen@enron.com,Re: Hello,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,,None,Let's shoot for Tuesday at 11:45.,greg.piper@enron.com,Allen-P,pallen.nsf,Phillip K Allen,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Greg Piper,<30922949.1075863688243.JavaMail.evans@thyme>,allen-p


In [0]:
emails_df['Subjcontent'] = emails_df['Subject'] + " " + emails_df['content']

## Text preprocessing
* Cleaning, density Compression
* Tokenization

In [42]:
p_stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

#Data cleaning
def email_cleaning(text):
    # clean and tokenize document string
    email_content = tokenizer.tokenize(text.lower()) 
    word_list = []
    for i in email_content:
        if (('http' not in i) and ('@' not in i) and ('<.*?>' not in i) and i.isalnum() and (not i in stop_words)):
            word_list += [i]
        
    return word_list 

#Data Pre-processing
def preprocessing(text):    
    # remove numbers
    number_tokens = [re.sub(r'[\d]', ' ', i) for i in text]
    number_tokens = ' '.join(number_tokens).split()
     # stem tokens
    processed_tokens = [p_stemmer.stem(i) for i in number_tokens]
    
    # lemmatize tokens
    #processed_tokens = [lemmatizer.lemmatize(token) for token in number_tokens]
    
    # remove empty
    length_tokens = [i for i in processed_tokens if len(i) > 1]
    return length_tokens

  # Test
preprocessing(email_cleaning('It is going to rain today. Vini sings nicely.'))

['go', 'rain', 'today', 'vini', 'sing', 'nice']

In [0]:
# get entire text documents converted into list of list of preprocessed tokens
def get_tokened_content(content_df):
  all_content_tokened = []

  j=0
  k=0
  
  for content in content_df:           
      #Data cleaning
      clean_content = email_cleaning(content)

      #Pre-processing
      processed_email = preprocessing(clean_content)

      # add tokens to list
      if processed_email:
          all_content_tokened.append(processed_email)
          j+=1

      k+=1

  print("Number of emails processed: ", k)
  print("Number of non-empty emails vectors: ", j)
  return all_content_tokened

In [16]:
all_content_tokened = get_tokened_content(emails_df.Subjcontent)

Number of emails processed:  96544
Number of non-empty emails vectors:  96063


## LDA model training

### Corpus and dictionary preparation

In [0]:
dictionary = Dictionary(list(all_content_tokened))
corpus = [dictionary.doc2bow(text) for text in all_content_tokened]

In [0]:
# LDA model on complete corpus
NUM_TOPICS = 4
lda = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30)

In [19]:
lda.print_topics()

[(0,
  '0.011*"market" + 0.008*"would" + 0.008*"enron" + 0.008*"power" + 0.006*"energi" + 0.005*"compani" + 0.005*"price" + 0.005*"new" + 0.004*"provid" + 0.004*"gener"'),
 (1,
  '0.016*"get" + 0.014*"go" + 0.012*"would" + 0.011*"know" + 0.010*"like" + 0.009*"think" + 0.009*"want" + 0.008*"good" + 0.008*"look" + 0.008*"time"'),
 (2,
  '0.031*"pm" + 0.023*"pleas" + 0.022*"enron" + 0.015*"mark" + 0.012*"forward" + 0.012*"intend" + 0.012*"meet" + 0.011*"john" + 0.009*"thank" + 0.008*"may"'),
 (3,
  '0.016*"pleas" + 0.012*"deal" + 0.010*"pm" + 0.010*"need" + 0.010*"agreement" + 0.010*"trade" + 0.009*"would" + 0.008*"know" + 0.007*"chang" + 0.007*"let"')]

### Focused Topic Analysis
* From previous analysis exercise(file:enron_email_analysis), we had found <b>2001</b> as the peak year with most email communications
* So, I have considered only 2001 year for focus on main topics occurred on that year.

In [0]:
emails_df['Date'] = pd.to_datetime(emails_df['Date'], infer_datetime_format=True)
year_content = emails_df[emails_df.Date.dt.year==2001].Subjcontent
year_tokened_content = get_tokened_content(year_content)

In [0]:
dictionary = Dictionary(list(year_tokened_content))
corpus = [dictionary.doc2bow(text) for text in all_content_tokened]
NUM_TOPICS = 10
year_lda = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30)

In [36]:
year_lda.print_topics()

[(0,
  '0.018*"enron" + 0.013*"market" + 0.013*"trade" + 0.010*"compani" + 0.009*"new" + 0.007*"busi" + 0.006*"would" + 0.005*"manag" + 0.005*"energi" + 0.005*"provid"'),
 (1,
  '0.021*"power" + 0.012*"would" + 0.011*"energi" + 0.010*"market" + 0.009*"electr" + 0.008*"california" + 0.008*"gener" + 0.007*"state" + 0.007*"ferc" + 0.007*"propos"'),
 (2,
  '0.012*"travel" + 0.011*"way" + 0.010*"reserv" + 0.009*"hotel" + 0.009*"tx" + 0.007*"citi" + 0.007*"de" + 0.007*"houston" + 0.007*"la" + 0.007*"feb"'),
 (3,
  '0.028*"meet" + 0.022*"pleas" + 0.016*"pm" + 0.013*"would" + 0.010*"call" + 0.010*"thank" + 0.009*"confer" + 0.008*"john" + 0.008*"group" + 0.008*"work"'),
 (4,
  '0.045*"intend" + 0.031*"enron" + 0.029*"may" + 0.026*"messag" + 0.025*"confidenti" + 0.024*"pleas" + 0.023*"recipi" + 0.022*"isda" + 0.021*"receiv" + 0.020*"contain"'),
 (5,
  '0.027*"get" + 0.022*"go" + 0.019*"would" + 0.018*"know" + 0.016*"think" + 0.016*"want" + 0.015*"like" + 0.012*"look" + 0.012*"good" + 0.011*"pm"'

In [0]:
#with open('year_lda.model', 'wb') as f:
year_lda.save('year_lda.model')

files.download('year_lda.model')

In [0]:
lda.save('lda.model')

files.download('lda.model')